In [26]:
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from algorithms.simulated_annealing import Annealing as SA
from tools.data_loader import get_data
from tools.measures import get_all_results
from tools.check_result import check, get_res

pd.options.display.max_rows = 100

# Лабораторная работа № 4

## A simulated annealing algorithm for manufacturing cell formation problem

Выполнили Куклин, Кислицына, Аросланкин.  
Здесь представлены результаты работы алгоритма Имитации отжига (далее SA) для решения CFP.  
Описание алгоритма и параметров можно найти в [статье](https://ir.nctu.edu.tw/bitstream/11536/9500/1/000253183700003.pdf).

### Эксперименты

Выкачиваем и формируем примеры задач, на которых будем проводить эксперименты.

In [30]:
files = os.listdir('./data/cfp_data')
files.sort()
cases = []
for file in files:
    cases.append(get_data(file))

20x20.txt  is done!
24x40.txt  is done!
30x50.txt  is done!
30x90.txt  is done!
37x53.txt  is done!


Параметры алгоритма:  
* **C** - стартовое количество кластеров
* **T0** - стартовое значение температуры
* **Tf** - предельное значение температуры (при достижение ее останавливаем итерацию алгоритма)  
* **alpha** - коэффициент снижения температуры с каждой итерацией
* **L** - длина Марковской цепи (кол-во итераций для внутреннего цикла)
* **D** - период выполнения exchange-move  
* **check** - ограничение на простаивание внутреннего цикла 
(ограничивает кол-во раз, когда соседнее полученное решение совпадает с предыдущим решением)

Теперь для каждого примера задачи запустим алгоритм с разными значениями параметров.   
Для каждого набора параметров высчитаем среднее время выполнения алгоритма (среднее из 10 итераций) 
(**mean_time**), покажем значение целевой функции для найденного решения (**efficacy**) и кол-во кластеров, 
соответствующие данному решению (**clusters**).   

Для данного эксперимента были взяты следующие значения параметров:  
* **C** = 2
* **T0** = {10, 30, 50}
* **Tf** = 0.002
* **alpha** = {0.7, 0.8, 0.9}
* **L** = {10, 30, 70}
* **D** = {6, 12, 18} 
* **check** = 4 

In [4]:
df = get_all_results(cases)

100%|██████████| 5/5 [1:37:52<00:00, 1174.54s/it]


Разберем каждый пример по-отдельности:

## 20x20

Таблицы, которые показывают результаты для всех сочетаний изменяемых параметров, снесены вниз, так как они большие.
Далее будут приведены и описаны выборки из этих таблиц для наглядности. 

Посмотрим, как меняется время работы SA и его результаты в зависимости от каждого изменяемого параметра.

#### Параметр D (период exchange-move):

In [7]:
df[(df.case == '20x20.txt') & (df.T0 == 50) & (df.alpha == 0.9) & (df.L == 70)][['T0', 'alpha', 'L', 'D', 'mean_time', 'efficacy', 'clusters']]

,T0,alpha,L,D,mean_time,efficacy,clusters
78,50,0.9,70,6,26.252254,0.424460,5
79,50,0.9,70,12,13.002107,0.403409,3
80,50,0.9,70,18,12.944870,0.403409,3


При прочих равных условиях при уменьшении параметра **D** увеличивается время работы алгоритма, но при этом повышается и оптимальность решения, что логично, потому что при меньшем **D** мы чаще делаем exchange-move, а значит чаще выпригиваем из локальных минимумов. 

#### Параметр L (кол-во итерации для внутреннего цикла):

In [8]:
df[(df.case == '20x20.txt') & (df.T0 == 50) & (df.alpha == 0.9) & (df.D == 6)][['T0', 'alpha', 'L', 'D', 'mean_time', 'efficacy', 'clusters']]


,T0,alpha,L,D,mean_time,efficacy,clusters
72,50,0.9,10,6,0.668669,0.42446,5
75,50,0.9,30,6,10.132343,0.42446,5
78,50,0.9,70,6,26.252254,0.42446,5


Увеличение **L** дает сильный прирост времени работы алгоритма.    
Как видно из общей таблицы, на решение, которое выдает SA **для данного примера**, влияет только параметр **D**. От изменения других параметров решение не меняется. 

#### Параметр alpha (коэффициент понижения температуры):

In [9]:
df[(df.case == '20x20.txt') & (df.T0 == 50) & (df.L == 70) & (df.D == 6)][['T0', 'alpha', 'L', 'D', 'mean_time', 'efficacy', 'clusters']]


,T0,alpha,L,D,mean_time,efficacy,clusters
60,50,0.7,70,6,16.223227,0.42446,5
69,50,0.8,70,6,24.646752,0.42446,5
78,50,0.9,70,6,26.252254,0.42446,5


Чем выше данный коэффициент, тем медленнее понижается температура, соответственно, время работы алгоритма больше.

#### Параметр T0 (начальная температура):

In [10]:
df[(df.case == '20x20.txt') & (df.alpha == 0.9) & (df.L == 70) & (df.D == 6)][['T0', 'alpha', 'L', 'D', 'mean_time', 'efficacy', 'clusters']]


,T0,alpha,L,D,mean_time,efficacy,clusters
24,10,0.9,70,6,18.132738,0.42446,5
51,30,0.9,70,6,18.605710,0.42446,5
78,50,0.9,70,6,26.252254,0.42446,5


При повышении начальной температуры, начиная с определенного значения, время работы тоже повышается.

## 24x40

In [15]:
print('Изменения D')
display(df[(df.case == '24x40.txt') & (df.T0 == 50) & (df.alpha == 0.8) & (df.L == 70)][['T0', 'alpha', 'L', 'D', 'mean_time', 'efficacy', 'clusters']])
print('Изменения L')
display(df[(df.case == '24x40.txt') & (df.T0 == 50) & (df.alpha == 0.9) & (df.D == 6)][['T0', 'alpha', 'L', 'D', 'mean_time', 'efficacy', 'clusters']])
print('Изменения alpha')
display(df[(df.case == '24x40.txt') & (df.T0 == 50) & (df.D == 6) & (df.L == 70)][['T0', 'alpha', 'L', 'D', 'mean_time', 'efficacy', 'clusters']])
print('Изменения T0')
display(df[(df.case == '24x40.txt') & (df.D == 6) & (df.alpha == 0.9) & (df.L == 70)][['T0', 'alpha', 'L', 'D', 'mean_time', 'efficacy', 'clusters']])


Изменения D


,T0,alpha,L,D,mean_time,efficacy,clusters
150,50,0.8,70,6,1.488221,0.364807,5
151,50,0.8,70,12,15.590912,0.363248,5
152,50,0.8,70,18,0.715981,0.363248,5


Изменения L


,T0,alpha,L,D,mean_time,efficacy,clusters
153,50,0.9,10,6,0.480656,0.364807,5
156,50,0.9,30,6,1.090023,0.364807,5
159,50,0.9,70,6,1.461864,0.364807,5


Изменения alpha


,T0,alpha,L,D,mean_time,efficacy,clusters
141,50,0.7,70,6,1.457013,0.364807,5
150,50,0.8,70,6,1.488221,0.364807,5
159,50,0.9,70,6,1.461864,0.364807,5


Изменения T0


,T0,alpha,L,D,mean_time,efficacy,clusters
105,10,0.9,70,6,1.459055,0.364807,5
132,30,0.9,70,6,1.502007,0.364807,5
159,50,0.9,70,6,1.461864,0.364807,5


Опять можно сделать вывод, что на результаты данного примера не особо влияют изменения параметров **L**, **alpha** и **T0**, а больше влияет параметр **D**. По времени работы можно сказать, что увеличение **L** увеличивает время, а вот **T0** и **alpha** практически не меняют в этом случае время работы. Для **D** = 12 время работы резко увеличивается, вероятно при таком периоде exchange-move алгоритм выскакиев из какого-то локального минимума и дольго после этого постепенно спускается.

## 30x50

In [24]:
print('Изменения D')
display(df[(df.case == '30x50.txt') & (df.T0 == 10) & (df.alpha == 0.7) & (df.L == 10)][['T0', 'alpha', 'L', 'D', 'mean_time', 'efficacy', 'clusters']])
print('Изменения L')
display(df[(df.case == '30x50.txt') & (df.T0 == 50) & (df.alpha == 0.9) & (df.D == 6)][['T0', 'alpha', 'L', 'D', 'mean_time', 'efficacy', 'clusters']])
print('Изменения alpha')
display(df[(df.case == '30x50.txt') & (df.T0 == 50) & (df.D == 6) & (df.L == 70)][['T0', 'alpha', 'L', 'D', 'mean_time', 'efficacy', 'clusters']])
print('Изменения T0')
display(df[(df.case == '30x50.txt') & (df.D == 6) & (df.alpha == 0.9) & (df.L == 70)][['T0', 'alpha', 'L', 'D', 'mean_time', 'efficacy', 'clusters']])


Изменения D


,T0,alpha,L,D,mean_time,efficacy,clusters
162,10,0.7,10,6,5.932926,0.487923,10
163,10,0.7,10,12,4.283469,0.487923,10
164,10,0.7,10,18,4.125273,0.487923,10


Изменения L


,T0,alpha,L,D,mean_time,efficacy,clusters
234,50,0.9,10,6,5.115053,0.487923,10
237,50,0.9,30,6,40.071485,0.487923,10
240,50,0.9,70,6,87.229098,0.487923,10


Изменения alpha


,T0,alpha,L,D,mean_time,efficacy,clusters
222,50,0.7,70,6,22.943145,0.487923,10
231,50,0.8,70,6,39.126161,0.487923,10
240,50,0.9,70,6,87.229098,0.487923,10


Изменения T0


,T0,alpha,L,D,mean_time,efficacy,clusters
186,10,0.9,70,6,72.378442,0.487923,10
213,30,0.9,70,6,82.458045,0.487923,10
240,50,0.9,70,6,87.229098,0.487923,10


По общей таблице можно заметить, что на результаты не повлияло изменение ни одного параметра. Поэтому рассмотрим изменение времени работы. При увеличение **L**, **alpha** и **T0** большой прирост по времени. На маленьком **D** работает дольше, чем на больших. 

## 30x90

In [21]:
print('Изменения D')
display(df[(df.case == '30x90.txt') & (df.T0 == 50) & (df.alpha == 0.8) & (df.L == 10)][['T0', 'alpha', 'L', 'D', 'mean_time', 'efficacy', 'clusters']])
print('Изменения L')
display(df[(df.case == '30x90.txt') & (df.T0 == 50) & (df.alpha == 0.8) & (df.D == 6)][['T0', 'alpha', 'L', 'D', 'mean_time', 'efficacy', 'clusters']])
print('Изменения alpha')
display(df[(df.case == '30x90.txt') & (df.T0 == 50) & (df.D == 6) & (df.L == 10)][['T0', 'alpha', 'L', 'D', 'mean_time', 'efficacy', 'clusters']])
print('Изменения T0')
display(df[(df.case == '30x90.txt') & (df.D == 6) & (df.alpha == 0.8) & (df.L == 10)][['T0', 'alpha', 'L', 'D', 'mean_time', 'efficacy', 'clusters']])


Изменения D


,T0,alpha,L,D,mean_time,efficacy,clusters
306,50,0.8,10,6,7.796792,0.387097,7
307,50,0.8,10,12,7.151341,0.378238,7
308,50,0.8,10,18,8.507877,0.378238,7


Изменения L


,T0,alpha,L,D,mean_time,efficacy,clusters
306,50,0.8,10,6,7.796792,0.387097,7
309,50,0.8,30,6,4.006400,0.281179,3
312,50,0.8,70,6,4.157988,0.281179,3


Изменения alpha


,T0,alpha,L,D,mean_time,efficacy,clusters
297,50,0.7,10,6,7.811417,0.359102,7
306,50,0.8,10,6,7.796792,0.387097,7
315,50,0.9,10,6,4.398141,0.281179,3


Изменения T0


,T0,alpha,L,D,mean_time,efficacy,clusters
252,10,0.8,10,6,2.721576,0.277162,3
279,30,0.8,10,6,3.033241,0.280460,3
306,50,0.8,10,6,7.796792,0.387097,7


В данном примере дела обстоят интереснее. Увеличение **T0** повышает как время, так и оптимальность результата. В противовес в первым трем примерам, увеличение параметров **L** и **alpha** уменьшает время работы, а **D** увеличивает. Посмотрим на послений пример и сделаем общие выводы по данным изменяемым параметрам.

## 37x53

In [27]:
print('Изменения D')
display(df[(df.case == '37x53.txt') & (df.T0 == 50) & (df.alpha == 0.8) & (df.L == 10)][['T0', 'alpha', 'L', 'D', 'mean_time', 'efficacy', 'clusters']])
print('Изменения L')
display(df[(df.case == '37x53.txt') & (df.T0 == 50) & (df.alpha == 0.8) & (df.D == 6)][['T0', 'alpha', 'L', 'D', 'mean_time', 'efficacy', 'clusters']])
print('Изменения alpha')
display(df[(df.case == '37x53.txt') & (df.T0 == 50) & (df.D == 6) & (df.L == 10)][['T0', 'alpha', 'L', 'D', 'mean_time', 'efficacy', 'clusters']])
print('Изменения T0')
display(df[(df.case == '37x53.txt') & (df.D == 6) & (df.alpha == 0.7) & (df.L == 10)][['T0', 'alpha', 'L', 'D', 'mean_time', 'efficacy', 'clusters']])


Изменения D


,T0,alpha,L,D,mean_time,efficacy,clusters
387,50,0.8,10,6,0.875844,0.604651,3
388,50,0.8,10,12,0.819429,0.604651,3
389,50,0.8,10,18,0.805068,0.600736,3


Изменения L


,T0,alpha,L,D,mean_time,efficacy,clusters
387,50,0.8,10,6,0.875844,0.604651,3
390,50,0.8,30,6,0.988686,0.604651,3
393,50,0.8,70,6,1.003060,0.604651,3


Изменения alpha


,T0,alpha,L,D,mean_time,efficacy,clusters
378,50,0.7,10,6,0.815036,0.604651,3
387,50,0.8,10,6,0.875844,0.604651,3
396,50,0.9,10,6,0.984605,0.604651,3


Изменения T0


,T0,alpha,L,D,mean_time,efficacy,clusters
324,10,0.7,10,6,0.715442,0.582267,3
351,30,0.7,10,6,0.764134,0.604651,3
378,50,0.7,10,6,0.815036,0.604651,3


Эталонный пример. Увеличение параметров **L**, **alpha** и **T0** увеличитает время работы, но и немного улучшает результаты. Уменьшение **D** увеличивает время работы и немного улучшает результаты.

***Общий вывод по параметрам D, L, alpha и T0:***  
Раз на раз не приходится. По большинству примеров можно заметить общую тенденцию, что увеличение значения параметров **L**, **alpha** и **T0** увеличивает время работы алгоритма и бывает немногим улучшает результаты. Изменение **D** влияет по-разному. 


Сравним время работы SA для решения этих 5 примеров между собой при одинаковых параметрах.

In [28]:
display(df[(df.T0 == 50) & (df.alpha == 0.8) & (df.L == 10) & (df.D == 6)][['case', 'T0', 'alpha', 'L', 'D', 'mean_time', 'efficacy', 'clusters']])


,case,T0,alpha,L,D,mean_time,efficacy,clusters
63,20x20.txt,50,0.8,10,6,1.425370,0.424460,5
144,24x40.txt,50,0.8,10,6,0.765054,0.364807,5
225,30x50.txt,50,0.8,10,6,5.463814,0.487923,10
306,30x90.txt,50,0.8,10,6,7.796792,0.387097,7
387,37x53.txt,50,0.8,10,6,0.875844,0.604651,3


От количества машин и деталей время особо не зависит. Оно зависит больше от количества кластеров, на которые следует разбивать: чем кластеров больше, соответственно тем дольше до них доходит алгоритм.

## Лучшие полученные результаты

Найдем лучшие решения, которые получились в предыдущем эксперименте, и для них лучшее время. 

In [20]:
ind = []
for case in ['20x20.txt', '24x40.txt', '30x50.txt', '30x90.txt', '37x53.txt']:
    m = df[df.case == case]['efficacy'].max()
    ind.append(df[(df.case == case) & (df.efficacy == m)]['mean_time'].idxmin())
df_sol = df.iloc[ind]
df_sol.index = range(5)
df_sol

,case,C,T0,Tf,alpha,L,D,check,mean_time,efficacy,clusters
0,20x20.txt,2,50,0.002,0.9,10,6,4,0.668669,0.424460,5
1,24x40.txt,2,10,0.002,0.7,10,18,4,1.232533,0.418848,7
2,30x50.txt,2,10,0.002,0.8,10,6,4,3.831539,0.487923,10
3,30x90.txt,2,50,0.002,0.8,10,6,4,7.796792,0.387097,7
4,37x53.txt,2,30,0.002,0.7,70,6,4,0.734502,0.604651,3


Но в ходе экспериментов было замечено, что для некоторых задач для получения более оптимального решения необходимо увеличить начальное количество кластеров.

И мы нашли следующие улучшения:

In [10]:
df_C = pd.read_csv('./data/results_С.csv')
df_C.drop('Unnamed: 0', 1, inplace=True)
ind = []
for case in ['24x40.txt', '30x90.txt',]:
    m = df_C[df_C.case == case]['efficacy'].max()
    ind.append(df_C[(df_C.case == case) & (df_C.efficacy == m)]['mean_time'].idxmin())
df_C.iloc[ind]

,case,C,T0,Tf,alpha,L,D,check,mean_time,efficacy,clusters
17,24x40.txt,10,50,0.002,0.7,70,18,4,2.242713,0.456376,11
33,30x90.txt,10,50,0.002,0.7,70,6,4,15.793779,0.399471,11


## Итоговые лучшие результаты
Решения, соответствующие данным результатам, находятся в [папке]()

In [29]:
df_sol.iloc[1] = df_C.iloc[17]
df_sol.iloc[3] = df_C.iloc[33]
df_sol

,case,C,T0,Tf,alpha,L,D,check,mean_time,efficacy,clusters
0,20x20.txt,2,50,0.002,0.9,10,6,4,0.668669,0.424460,5
1,24x40.txt,10,50,0.002,0.7,70,18,4,2.242713,0.456376,11
2,30x50.txt,2,10,0.002,0.8,10,6,4,3.831539,0.487923,10
3,30x90.txt,10,50,0.002,0.7,70,6,4,15.793779,0.399471,11
4,37x53.txt,2,30,0.002,0.7,70,6,4,0.734502,0.604651,3


## Таблицы со всеми наборами изменяемых параметров для каждого отдельного примера

In [72]:
print('20x20')
display(df[df.case == '20x20.txt'][['T0', 'alpha', 'L', 'D', 'mean_time', 'efficacy', 'clusters']])
print('\n24x40')
display(df[df.case == '24x40.txt'][['T0', 'alpha', 'L', 'D', 'mean_time', 'efficacy', 'clusters']])
print('\n30x50')
display(df[df.case == '30x50.txt'][['T0', 'alpha', 'L', 'D', 'mean_time', 'efficacy', 'clusters']])
print('\n30x90')
display(df[df.case == '30x90.txt'][['T0', 'alpha', 'L', 'D', 'mean_time', 'efficacy', 'clusters']])
print('\n37x53')
display(df[df.case == '37x53.txt'][['T0', 'alpha', 'L', 'D', 'mean_time', 'efficacy', 'clusters']])


20x20


,T0,alpha,L,D,mean_time,efficacy,clusters
0,10,0.7,10,6,0.630770,0.420290,5
1,10,0.7,10,12,0.325325,0.403409,3
2,10,0.7,10,18,0.353506,0.403409,3
3,10,0.7,30,6,5.809688,0.424460,5
4,10,0.7,30,12,1.190165,0.403409,3
5,10,0.7,30,18,1.285646,0.403409,3
6,10,0.7,70,6,10.409518,0.424460,5
7,10,0.7,70,12,2.637017,0.403409,3
8,10,0.7,70,18,2.637063,0.403409,3
9,10,0.8,10,6,0.756210,0.420290,5



24x40


,T0,alpha,L,D,mean_time,efficacy,clusters
81,10,0.7,10,6,1.016158,0.364807,5
82,10,0.7,10,12,0.750260,0.363248,5
83,10,0.7,10,18,1.232533,0.418848,7
84,10,0.7,30,6,1.035369,0.364807,5
85,10,0.7,30,12,2.735826,0.363248,5
86,10,0.7,30,18,0.726663,0.363248,5
87,10,0.7,70,6,1.482354,0.364807,5
88,10,0.7,70,12,5.431609,0.363248,5
89,10,0.7,70,18,0.802608,0.363248,5
90,10,0.8,10,6,1.281317,0.364807,5



30x50


,T0,alpha,L,D,mean_time,efficacy,clusters
162,10,0.7,10,6,5.932926,0.487923,10
163,10,0.7,10,12,4.283469,0.487923,10
164,10,0.7,10,18,4.125273,0.487923,10
165,10,0.7,30,6,10.378035,0.487923,10
166,10,0.7,30,12,6.654758,0.487923,10
167,10,0.7,30,18,7.522489,0.487923,10
168,10,0.7,70,6,18.151430,0.487923,10
169,10,0.7,70,12,9.993583,0.487923,10
170,10,0.7,70,18,11.074350,0.487923,10
171,10,0.8,10,6,3.831539,0.487923,10



30x90


,T0,alpha,L,D,mean_time,efficacy,clusters
243,10,0.7,10,6,5.926540,0.346633,7
244,10,0.7,10,12,7.233370,0.355000,7
245,10,0.7,10,18,6.078207,0.349127,7
246,10,0.7,30,6,7.393195,0.346633,7
247,10,0.7,30,12,7.746685,0.355000,7
248,10,0.7,30,18,7.756730,0.349127,7
249,10,0.7,70,6,7.806093,0.346633,7
250,10,0.7,70,12,7.874278,0.355000,7
251,10,0.7,70,18,7.758870,0.349127,7
252,10,0.8,10,6,2.721576,0.277162,3



37x53


,T0,alpha,L,D,mean_time,efficacy,clusters
324,10,0.7,10,6,0.715442,0.582267,3
325,10,0.7,10,12,0.626941,0.588665,3
326,10,0.7,10,18,0.641486,0.596330,3
327,10,0.7,30,6,0.710255,0.582267,3
328,10,0.7,30,12,0.676267,0.588665,3
329,10,0.7,30,18,0.639849,0.596330,3
330,10,0.7,70,6,0.674380,0.582267,3
331,10,0.7,70,12,0.656585,0.588665,3
332,10,0.7,70,18,0.642101,0.596330,3
333,10,0.8,10,6,0.920107,0.604651,3
